In [1]:
import pandas as pd
import numpy
import pickle
import numpy as np
from scipy import stats
from collections import Counter

# Find number of images in each class

In [3]:
train_csv = pd.read_csv('train.csv').drop(['Unnamed: 0'], axis=1)
train_csv.head()

,Image,Label
0,19567,9
1,29993,6 4
2,17186,1 4
3,29600,6 2
4,701,3 4


In [4]:
all_label = []
for label in train_csv.Label:
     for l in str(label).split(' '):
        all_label.append(int(l))
from collections import Counter
Counter(all_label)

Counter({9: 1278,
         6: 5711,
         4: 9066,
         1: 1752,
         2: 2542,
         3: 1977,
         7: 2629,
         0: 2088,
         8: 1037,
         5: 1109})

## Fill NA with mode (class 4)
Didn't use this method in the submission.

In [5]:
def check_na(table, filled_table, is_max=False):
    print(len(table[table.Label.isnull()]))
    print(len(filled_table[filled_table.Label.isnull()]))
    new_table = table.copy()
    
    new_table = table.merge(filled_table, on='Image', how='left', suffixes=('', '_b'))
    
    if is_max==False:
        filled = filled_table.Label
    else:
        filled = 4
        new_table['Label'] = new_table.fillna(new_table.Label_b)
    print(f"after{len(new_table[new_table.Label.isnull()])}")
    
    return new_table[['Image', 'Label']]

def ensem(s):
    list_s = s.split(' ')
    dups = [gene for gene, cnt in Counter(list_s).items() if cnt >= 2]
    if len(dups) > 0:
        res = ' '.join(dups)
    else:
        res = list_s[0]
        for pred in list_s:
            if label_dict[pred] > label_dict[res]:
                res = pred
    return res

In [7]:
# ensemble.Label = (ensemble.Label + ' '+ c.Label + ' '+ a.Label).str.strip()
# ensemble = ensemble.fillna('4')

# ensemble.Label = ensemble.Label.map(lambda x: ensem(x))
# ensemble

## Generating submission files start here

In [28]:
import pandas as pd
## Set the threshold to 0.5 to predict classes
def e_to_double(num_str):
    return [float(s.strip()) for s in num_str.strip('[]').split(' ') if s.strip() != '']

def decode_target(target, text_labels=False, threshold=0.5):
    result = []
    for i, x in enumerate(target):
        if (x >= threshold):
            if text_labels:
                result.append(labels[i] + "(" + str(i) + ")")
            else:
                result.append(str(i))
    return ' '.join(result)


def string_to_list(num_str):
    return [s.strip() for s in num_str[1:-1].split(',')]

def convert_to_float(tensor_str):
    return [float(i.replace('tensor', '').replace('(', '').replace(')', '')) for i in tensor_str]

def fill_na_with_threshold(df, th, argmax=False):
    print(f"#missing classes before th = {len(df[df.Label == ''])}")
    if argmax: 
        df['Label'] = np.where(df.Label == '', df.avg.apply(lambda x: np.argmax(x)), en.Label)
    else:
        df['Label'] = np.where(df.Label == '', df.avg.apply(lambda x: decode_target(x, threshold=th)), en.Label)
    print(f"#missing classes after th = {len(df[df.Label == ''])}")

In [29]:
df = pd.read_csv('pred_data/test_tf_efficientb1_submission_0.8246.csv')
df2 = pd.read_csv('pred_data/efficientb1_submission_0.8209.csv')
df3 = pd.read_csv('pred_data/efficientb2_submission_0.8_2jul.csv')
df4 = pd.read_csv('pred_data/efficientb2_submission_0.80.csv')
df5 = pd.read_csv('pred_data/efficientb1_0.86_2jul.csv')

# df6 = pd.read_csv('../efficientb1_submission_0.8201.csv')
# df6.head()

In [30]:
df['probs'] = df.Label.apply(lambda x: e_to_double(x))
df['Label'] = df.probs.apply(lambda x: decode_target(x))

df2['probs'] = df2.Label.apply(lambda x: e_to_double(x))
df2['Label'] = df2.probs.apply(lambda x: decode_target(x))

df3['probs'] = df3.Label.apply(lambda x: e_to_double(x))
df3['Label'] = df3.probs.apply(lambda x: decode_target(x))

df4['probs'] = df4.Label.apply(lambda x: e_to_double(x))
df4['Label'] = df4.probs.apply(lambda x: decode_target(x))

df5['probs'] = df5.Label.apply(lambda x: e_to_double(x))
df5['Label'] = df5.probs.apply(lambda x: decode_target(x))

#### There is one file below that I forgot to convert tensor to numpy so I had to modify a bit.


In [31]:
prob_df = pd.read_csv('pred_data/efficientb1_submission_tl0.1191_vl0.1590_vs0.8118.csv')
prob_df['probs'] = prob_df.Label.apply(lambda x: string_to_list(x))
prob_df['probs'] = prob_df.probs.apply(lambda x: convert_to_float(x))
prob_df['Label'] = prob_df.probs.apply(lambda x: decode_target(x))

In [33]:
join_df = prob_df[['Image','probs']].merge(df[['Image','probs']], on='Image', how='inner', suffixes=('', '_a'))
join_df = join_df[['Image','probs','probs_a']].merge(df2[['Image','probs']], on='Image', how='inner', suffixes=('', '_8209'))
join_df = join_df[['Image','probs','probs_a', 'probs_8209']].merge(df3[['Image','probs']], on='Image', how='inner', suffixes=('', '_8215'))
join_df = join_df[['Image','probs','probs_a', 'probs_8209', 'probs_8215']].merge(df4[['Image','probs']], on='Image', how='inner', suffixes=('', '_80'))
join_df = join_df.merge(df5[['Image','probs']], on='Image', how='inner', suffixes=('', '_86'))

## Ensemble 6 predictions, double weighted to the first one 

In [34]:
join_df.head()

,Image,probs,probs_a,probs_8209,probs_8215,probs_80,probs_86
0,24117,"[0.0004, 0.0155, 0.0024, 0.2518, 0.95, 0.0009,...","[9.5058465e-05, 0.014020162, 0.00037502506, 0....","[0.01641696, 0.01638814, 0.00362593, 0.3331707...","[5.7942005e-05, 0.0050297314, 9.1644088e-06, 0...","[0.016672028, 0.0018953624, 0.00018162145, 0.0...","[0.000479539216, 0.00156817841, 0.000136659233..."
1,15322,"[2.4524e-07, 0.2182, 0.0005, 1.1602e-07, 0.988...","[0.0001791089, 0.18194675, 0.013605975, 7.4465...","[2.8430513e-06, 0.19838126, 0.000112322, 1.627...","[0.00021067445, 0.106468789, 0.003323121, 1.36...","[3.5563509e-05, 3.394056e-05, 9.4118752e-07, 6...","[0.00024917573, 0.041026946, 0.0051044147, 1.0..."
2,14546,"[0.0032, 0.0002, 0.1267, 0.0011, 0.001, 0.0002...","[0.002290464, 0.0031467017, 0.0091130901, 0.00...","[0.001825957, 0.00045198938, 0.21213529, 0.002...","[0.00031822547, 0.0012825869, 0.024267085, 0.0...","[0.00014495358, 3.9940132e-05, 0.0096416082, 0...","[0.00018258826, 0.0019051344, 0.00075059681, 0..."
3,8079,"[0.7786, 0.0006, 0.012, 0.0145, 0.0071, 0.0005...","[0.26363873, 0.0010697065, 0.00096246751, 0.74...","[0.67908257, 0.0017855426, 8.8504312e-05, 0.32...","[0.33807683, 0.00053502648, 9.0419009e-05, 0.3...","[0.22237635, 8.3819366e-05, 7.6223723e-06, 0.0...","[0.013339059, 0.0022857406, 0.00015843546, 0.4..."
4,13192,"[0.1838, 0.0495, 0.1327, 0.6854, 0.8478, 0.173...","[0.018926281, 0.01482162, 0.0067124637, 0.7886...","[0.2977565, 0.09708337, 0.03678794, 0.5183007,...","[0.0012364932, 0.00039242712, 0.01967275, 0.99...","[0.010964317, 0.0090751778, 0.0017462507, 0.81...","[0.0081615355, 0.00052142725, 0.0063790623, 0...."


In [35]:
import numpy as np

lst_add = []
for n, row in join_df.iterrows():
    add_prob = (np.array(row['probs']) * 2) + np.array(row['probs_8209']) + np.array(row['probs_a']) + np.array(row['probs_80']) + np.array(row['probs_8215']) + np.array(row['probs_86'])
    lst_add.append(list(add_prob / 6))

In [36]:
join_df['avg'] = lst_add
join_df['Label'] = join_df.avg.apply(lambda x: decode_target(x))

en = join_df[['Image', 'Label', 'avg']]
en.head()

,Image,Label,avg
0,24117,4 8,"[0.005753587947666666, 0.011650262368333332, 0..."
1,15322,4,"[0.00011297602004999999, 0.16070961426, 0.0038..."
2,14546,6,"[0.0018603647183333336, 0.0012043920853333334,..."
3,8079,0 6,"[0.5122855898333333, 0.001159972591, 0.0042179..."
4,13192,3 4,"[0.11744085444999998, 0.03681567036166667, 0.0..."


## Changes threshold to fill missing labels

In [37]:
ensemble_df = fill_na_with_threshold(en, 0.46)
ensemble_df = fill_na_with_threshold(en, 0.445)
ensemble_df = fill_na_with_threshold(en, 1, argmax=True)

#missing classes before th = 122
#missing classes after th = 82
#missing classes before th = 82
#missing classes after th = 73
#missing classes before th = 73
#missing classes after th = 0


/Users/mimi/anaconda3/envs/audio/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mimi/anaconda3/envs/audio/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [38]:
final_df = en[['Image', 'Label']]
final_df.head()

,Image,Label
0,24117,4 8
1,15322,4
2,14546,6
3,8079,0 6
4,13192,3 4


In [117]:
# final_df.to_csv('path/final_submission.csv',index=False)